# 7.1 程序调试

程序能一次写完并正常运行的概率很小，基本不超过1%。总会有各种各样的bug需要修正，有的bug很简单，看看错误信息就知道，有的bug很复杂，我们需要知道出错时，哪些变量的值是正确的，哪些变量的值是错误的，因此，需要一整套调试程序的手段来修复bug。

## 7.1 print

第一种方法简单直接粗暴有效，就是用print把可能有问题的变量打印出来看看：

In [1]:
# err.py
def foo(s):
  n = int(s)
  print('>>> n = %d' % n)
  return 10 / n
 
def main():
  foo('0')
 
main()

>>> n = 0


ZeroDivisionError: division by zero

执行后在输出中查找打印的变量值：
   
    >>> n = 0

## 7.2 断言 assert

凡是用print来辅助查看的地方，都可以用断言（assert）来替代：

In [3]:
# err.py
def foo(s):
  n = int(s)
  assert(n != 0, 'n is zero!')
  return 10 / n
 
def main():
  foo('0')

<ipython-input-3-31b241b88f37>:4: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(n != 0, 'n is zero!')


assert的意思是，表达式n != 0应该是True，否则，后面的代码就会出错。

如果断言失败，assert语句本身就会抛出AssertionError

程序中如果到处充斥着assert，和print相比也好不到哪去。不过，启动Python解释器时可以用-O参数来关闭assert

python -O err.py

In [6]:
%%writefile err.py
def foo(s):
  n = int(s)
  assert(n != 0, 'n is zero!')
  return 10 / n
 
def main():
  foo('0')

Writing err.py


## 7.3 logging

把print替换为logging是第3种方式，和assert比，logging不会抛出错误，而且可以输出到文件：

### 7.3.1 简单将日志打印到屏幕

In [15]:
import logging  

logging.debug('debug message')  
logging.info('info message')  
logging.warning('warning message')  
logging.error('error message')  
logging.critical('critical message')  

ERROR:root:error message
CRITICAL:root:critical message


可见，默认情况下python的logging模块将日志打印到了标准输出中，且只显示了大于等于WARNING级别的日志，

这说明默认的日志级别设置为WARNING（日志级别等级CRITICAL > ERROR > WARNING > INFO > DEBUG > NOTSET）

默认的日志格式为:
    
     日志级别：Logger名称：用户输出消息。

### 7.3.2 灵活配置日志级别，日志格式，输出位置

In [23]:
%%writefile err.py
import logging  
logging.basicConfig(level=logging.DEBUG,  
                    format='%(asctime)s %(filename)s[line:%(lineno)d] %(levelname)s %(message)s',  
                    datefmt='%a, %d %b %Y %H:%M:%S',  
                    filename='test.log',  
                    filemode='w')  
  
logging.debug('debug message')  
logging.info('info message')  
logging.warning('warning message')  
logging.error('error message')  
logging.critical('critical message')  

Overwriting err.py


In [24]:
%run err.py  # 在终端运行

ERROR:root:error message
CRITICAL:root:critical message


In [13]:
%%writefile err.py
# err.py
import logging
logging.basicConfig(level=logging.INFO)

s = '0'
n = int(s)
logging.info('n = %d' % n)
print(10 / n)

Overwriting err.py


    在logging.basicConfig()函数中可通过具体参数来更改logging模块默认行为，可用参数有
    filename：用指定的文件名创建FiledHandler（后边会具体讲解handler的概念），这样日志会被存储在指定的文件中。
    filemode：文件打开方式，在指定了filename时使用这个参数，默认值为“a”还可指定为“w”。
    format：指定handler使用的日志显示格式。 
    datefmt：指定日期时间格式。 
    level：设置rootlogger（后边会讲解具体概念）的日志级别 
    stream：用指定的stream创建StreamHandler。可以指定输出到sys.stderr,sys.stdout或者文件，默认为sys.stderr。若同时列出了filename和stream两个参数，则stream参数会被忽略。

    format参数中可能用到的格式化串：
    %(name)s Logger的名字
    %(levelno)s 数字形式的日志级别
    %(levelname)s 文本形式的日志级别
    %(pathname)s 调用日志输出函数的模块的完整路径名，可能没有
    %(filename)s 调用日志输出函数的模块的文件名
    %(module)s 调用日志输出函数的模块名
    %(funcName)s 调用日志输出函数的函数名
    %(lineno)d 调用日志输出函数的语句所在的代码行
    %(created)f 当前时间，用UNIX标准的表示时间的浮 点数表示
    %(relativeCreated)d 输出日志信息时的，自Logger创建以 来的毫秒数
    %(asctime)s 字符串形式的当前时间。默认格式是 “2003-07-08 16:49:45,896”。逗号后面的是毫秒
    %(thread)d 线程ID。可能没有
    %(threadName)s 线程名。可能没有
    %(process)d 进程ID。可能没有
    %(message)s用户输出的消息

## 7.4 pdb

第4种方式是启动Python的调试器pdb，让程序以单步方式运行，可以随时查看运行状态。我们先准备好程序

In [26]:
%%writefile err.py
# err.py
s = '0'
n = int(s)
print(10 / n)

Overwriting err.py


然后在终端启动

    python -m pdb err.py

以参数-m pdb启动后，pdb定位到下一步要执行的代码-> s = '0'。输入命令l来查看代码：

    (Pdb) l
     1   # err.py
     2 -> s = '0'
     3   n = int(s)
     4   print 10 / n
    [EOF]
    
输入命令n可以单步执行代码：

    (Pdb) n
    
任何时候都可以输入命令p 变量名来查看变量：
    
    (Pdb) p s
    '0'
    (Pdb) p n
    0
    
输入命令q结束调试，退出程序：    
    
    (Pdb) q

这种通过pdb在命令行调试的方法理论上是万能的，但实在是太麻烦了，如果有一千行代码，要运行到第999行得敲多少命令啊。还好，我们还有另一种调试方法。

**pdb.set_trace()**

这个方法也是用pdb，但是不需要单步执行，我们只需要import pdb，然后，在可能出错的地方放一个pdb.set_trace()，就可以设置一个断点：

In [2]:
%%writefile err.py
#encoding=utf-8
#err.py
import pdb
 
s = '0'
n = int(s)
pdb.set_trace() # 运行到这里会自动暂停
print(10 / n)

Overwriting err.py


在终端运行代码，程序会自动在pdb.set_trace()暂停并进入pdb调试环境，可以用命令p查看变量，或者用命令c继续运行

    python err.py 
    
这个方式比直接启动pdb单步调试效率要高很多，但也高不到哪去。

运行脚本后执行响应的命令进行调试:

    Documented commands (type help <topic>):
    ========================================
    !: 后接语句可以直接改变变量的值

    a/args: 打印当前函数的参数

    alias: 给一段执行程序取别名(alias [name [command]])

    b/break: 设置断点(b [([filename:]lineno | function) [, condition]])

    c/cont/continue:  继续执行脚本

    cl/clear: 清除断点(cl [filename:lineno | bpnumber [bpnumber ...]])

    commands:

    disable: 禁用启用的断点(disable [bpnumber [bpnumber ...]])

    d/down: 移动到下一层堆栈(d [count])

    debug:

    display:

    enable: 启用禁用的断点(enable [bpnumber [bpnumber ...]])

    h/?/help: 打印帮助信息(h [command])

    ignore:

    interact:

    j/jump: 跳转到指定行

    l/list: 查看当前代码块

    ll: 查看完整源代码

    longlist:

    n/next: 执行下一行代码

    p/pp: 打印变量或表达式

    q/quit/exit:  终止执行并退出调试

    r/return: 执行脚本直到从当前函数返回

    run/restart:  重新开始调试

    rv/retval:

    s/step: 进入函数

    source:

    u/up: 移动到上一层堆栈(u [count])

    undisplay:

    unalias: 删除别名(unalias name)

    unt/until:

    w/bt/where:  打印堆栈轨迹

    whatis:

    Miscellaneous help topics:
    ==========================
    exec  pdb



更具体的信息可通过官方文档进行查看: [pdb文档](https://docs.python.org/3/library/pdb.html)

## 7.5 IDE

如果要比较爽地设置断点、单步执行，就需要一个支持调试功能的IDE。目前比较好的Python IDE有PyCharm：

http://www.jetbrains.com/pycharm/

另外，Eclipse加上pydev插件也可以调试Python程序。